## Limpieza

In [61]:
import pandas as pd

# Cargar la hoja 2 del archivo de Excel
file_path = "Planilla de pacientes_CORREGIDO.xlsx"
df = pd.read_excel(file_path, sheet_name=2, header=None)

# Eliminar las filas que no sirven
df = df.drop([0, 2, 3, 4], axis=0)

# Eliminar las columnas 'parametro', 'US', una vacía, 'date', 'start time' y 'end time'
df = df.drop([0, 1, 15, 18, 33, 34, 35], axis=1)

# Redefinir los nombres de las columnas con la nueva primera fila
df.columns = df.iloc[0]
df = df[1:]  # Eliminar la fila ahora redundante usada como encabezado

# Limpiar los espacios en blanco en la columna "Densidad" si existe
if "Densidad" in df.columns:
    df["Densidad"] = df["Densidad"].str.replace(" ", "", regex=True)

# Identificar las columnas numéricas (excluyendo las categóricas y de tiempo)
non_numercial_columns = ["Genero", "Densidad", "U/S Time", "Total Long. Power On Time", "Tot. TA On Time", "date", "start time", "end time", "Software Version"]
numeric_columns = df.columns.difference(non_numercial_columns)

# Reemplazar los puntos por comas en todas las columnas que sean relevantes
df = df.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

# Guardar el archivo limpio
df.to_excel("Planilla de pacientes limpio.xlsx", index=False)

# Mostrar una vista previa del DataFrame
print(df.head(3))


1  Edad Genero     AL    LT   ACD    WTW  CCT     K1     K2 Densidad  ...  \
5    77      M   23.7  5.49  2.53  11.78  538  44.83  45.63       N3  ...   
6    75      F   23.7   3.9  3.47  11.38  535  44.77  45.85   N3SCP3  ...   
7    66      F  23.15  4.62  3.44  12.12  527   44.1  44.98     N1C2  ...   

1  Est Fluid Aspi  Avg TA  Aver. TA(FP3)  Tot. TA On Time  \
5            68.0    19.1           12.9          00.50.3   
6            59.0    19.9           13.8          00.53.5   
7            61.0    18.1           12.5           1.02.8   

1  Equiv Aver. TA (FP3) Total Long. Power On Time  \
5                   5.2                  00:00:24   
6                   5.5                  00:00:23   
7                   5.0                  00:00:28   

1 Equiv. Aver. Ultrasonic Power (FP3) Aver. Long Power  \
5                                10.7             17.0   
6                                12.0             25.0   
7                                10.5             17.9   



C:\Users\Gonza\AppData\Local\Temp\ipykernel_3588\3243408118.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


In [62]:
# Chequeo de valores nulos
null_counts = df.isnull().sum()

# Distribución de los datos
# Convertir columnas no numéricas para análisis si es posible
numeric_df = df.apply(pd.to_numeric, errors='coerce')

# Obtener estadísticas descriptivas
data_distribution = numeric_df.describe()

# Mostrar resultados
print("Conteo de valores nulos por columna:")
print(null_counts)
print("\nDistribución de los datos:")
print(data_distribution)



Conteo de valores nulos por columna:
1
Edad                                    0
Genero                                  0
AL                                      0
LT                                      0
ACD                                     0
WTW                                     0
CCT                                     0
K1                                      0
K2                                      0
Densidad                                0
FR                                      0
IOP                                     0
Vacio                                   0
SatIs Pac                               0
Satis Cir                               0
Case Time                              11
CDE                                    11
U/S Time                               11
ASM Act                                11
Asp Time                               11
Est Fluid Aspi                         11
Avg TA                                 11
Aver. TA(FP3)                        

---

## Modelado con datos del paciente y biometricos

In [63]:
df = pd.read_excel('Planilla de pacientes limpio.xlsx', header=0)
df_simplificado = df[['Edad', 'Genero', 'AL', 'LT', 'ACD', 'WTW', 'CCT', 'K1', 'K2', 'Densidad', 'FR', 'IOP', 'Vacio']]
print(df_simplificado.head(10))

   Edad Genero     AL    LT   ACD    WTW  CCT     K1     K2 Densidad  FR  IOP  \
0    77      M  23.70  5.49  2.53  11.78  538  44.83  45.63       N3  35   30   
1    75      F  23.70  3.90  3.47  11.38  535  44.77  45.85   N3SCP3  35   30   
2    66      F  23.15  4.62  3.44  12.12  527  44.10  44.98     N1C2  35   30   
3    74      F  24.50  4.58  3.00  12.30  557  40.79  41.31       N4  35   30   
4    73      F  22.69  4.90  2.75  11.83  559  42.17  43.02     N1C2  35   30   
5    70      F  29.76  4.45  3.79  12.78  534  40.75  42.24     N1C3  30   20   
6    76      M  25.31  4.20  3.67  12.17  490  43.06  44.39     N2C2  35   30   
7    67      F  23.65  4.14  3.97  12.82  537  44.49  44.69   N3SCP3  35   30   
8    75      F  22.58  4.27  3.38  12.03  588  44.51  44.83       C4  35   30   
9    85      F  23.64  5.15  3.02  12.26  507  43.38  44.87     N4C4  35   30   

   Vacio  
0    350  
1    350  
2    350  
3    350  
4    350  
5    350  
6    350  
7    350  
8    350 

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

data = df_simplificado

# Separar características y objetivos
X = data.drop(columns=["FR", "IOP", "Vacio"])
y = data[["FR", "IOP", "Vacio"]]

# Identificar tipos de columnas
numeric_features = ["Edad", "AL", "LT", "ACD", "WTW", "CCT", "K1", "K2"]
categorical_features = ["Genero", "Densidad"]

# Crear un transformador para preprocesar las columnas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        ("cat", OneHotEncoder(), categorical_features),
    ]
)

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el pipeline con preprocesamiento y modelo
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entrenar el modelo con el pipeline
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['Edad', 'AL', 'LT', 'ACD',
                                                   'WTW', 'CCT', 'K1', 'K2']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Genero', 'Densidad'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [65]:
# Predecir en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular el error cuadrático medio para cada objetivo
mse_fr = mean_squared_error(y_test["FR"], y_pred[:, 0])
mse_iop = mean_squared_error(y_test["IOP"], y_pred[:, 1])
mse_vacio = mean_squared_error(y_test["Vacio"], y_pred[:, 2])

print(f"Mean Squared Error for FR: {mse_fr}")
print(f"Mean Squared Error for IOP: {mse_iop}")
print(f"Mean Squared Error for Vacio: {mse_vacio}")


ValueError: Found unknown categories ['N1C3'] in column 1 during transform

In [68]:
print(len(df['Densidad'].unique()))


25


#### Problema! Tenemos 25 categorias distintas de Densidad. Por lo que cuando usas OneHotEncoder, 25 categorías se convertirán en 25 columnas, lo cual puede diluir la información útil y sobreajustar el modelo. Las categorías con pocos datos contribuyen más al ruido que al aprendizaje, dificultando la generalización del modelo.